In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidymodels’ was built under R version 4.1.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.2.0 ──

✔ broom        0.7.12     ✔ rsample      0.1.1 
✔ dials        0.1.0      ✔ tune         0.2.0 
✔ infer        1.0.0      ✔ workflows    0.2.6 
✔ modeldata   

## Study of the effects of age on various statistics for position players in the last 12 years

In [40]:
data <- read.csv("downloads/FanGraphs Leaderboard (5).csv") 
head(data)

,Season,Name,Team,Age,G,PA,HR,R,RBI,SB,⋯,OBP,SLG,wOBA,xwOBA,wRC.,BsR,Off,Def,WAR,playerid
,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<lgl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,2020,Juan Soto,WSN,21,47,196,13,39,37,6,⋯,0.490,0.695,0.478,NA,202,-0.5,25.0,-3.9,2.6,20123
2,2015,Bryce Harper,WSN,22,153,654,42,118,99,6,⋯,0.460,0.649,0.461,NA,197,3.6,77.7,-10.2,9.3,11579
3,2020,Freddie Freeman,ATL,30,60,262,13,51,53,2,⋯,0.462,0.640,0.456,NA,186,-0.3,28.4,-3.0,3.2,5361
4,2013,Miguel Cabrera,DET,30,148,652,44,103,137,3,⋯,0.442,0.636,0.455,NA,193,-6.0,62.6,-4.0,8.6,1744
5,2018,Mookie Betts,BOS,25,136,614,32,129,80,30,⋯,0.438,0.640,0.449,NA,185,6.9,69.3,12.9,10.6,13611
6,2018,Mike Trout,LAA,26,140,608,39,101,79,24,⋯,0.460,0.628,0.447,NA,188,5.0,69.2,2.5,9.5,10155


In [6]:
colnames(data)

[1] "Season"   "Name"     "Team"     "Age"      "G"        "PA"      
 [7] "HR"       "R"        "RBI"      "SB"       "BB."      "K."      
[13] "ISO"      "BABIP"    "AVG"      "OBP"      "SLG"      "wOBA"    
[19] "xwOBA"    "wRC."     "BsR"      "Off"      "Def"      "WAR"     
[25] "playerid"

In [26]:
length(which(data$Age == 19))
length(which(data$Age == 20))
length(which(data$Age == 21))
length(which(data$Age == 22))
length(which(data$Age == 23))
length(which(data$Age == 24))
length(which(data$Age == 25))
length(which(data$Age == 26))
length(which(data$Age == 27))
length(which(data$Age == 28))
length(which(data$Age == 29))
length(which(data$Age == 30))
length(which(data$Age == 31))
length(which(data$Age == 32))
length(which(data$Age == 33))
length(which(data$Age == 34))
length(which(data$Age == 35))
length(which(data$Age == 36))
length(which(data$Age == 37))
length(which(data$Age == 38))
length(which(data$Age == 39))
length(which(data$Age == 40))
length(which(data$Age == 41))

## VERY small sample size from 19-21 and from 38-41.. these results do not contain much significance

[1] 1

[1] 6

[1] 17

[1] 40

[1] 74

...

[1] 25

[1] 10

[1] 10

[1] 4

[1] 1

In [24]:
num_rows <- data %>% 
group_by(Age) 
num_rows

Season,Name,Team,Age,G,PA,HR,R,RBI,SB,⋯,OBP,SLG,wOBA,xwOBA,wRC.,BsR,Off,Def,WAR,playerid
<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<lgl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020,Juan Soto,WSN,21,47,196,13,39,37,6,⋯,0.490,0.695,0.478,NA,202,-0.5,25.0,-3.9,2.6,20123
2015,Bryce Harper,WSN,22,153,654,42,118,99,6,⋯,0.460,0.649,0.461,NA,197,3.6,77.7,-10.2,9.3,11579
2020,Freddie Freeman,ATL,30,60,262,13,51,53,2,⋯,0.462,0.640,0.456,NA,186,-0.3,28.4,-3.0,3.2,5361
2013,Miguel Cabrera,DET,30,148,652,44,103,137,3,⋯,0.442,0.636,0.455,NA,193,-6.0,62.6,-4.0,8.6,1744
2018,Mookie Betts,BOS,25,136,614,32,129,80,30,⋯,0.438,0.640,0.449,NA,185,6.9,69.3,12.9,10.6,13611
2018,Mike Trout,LAA,26,140,608,39,101,79,24,⋯,0.460,0.628,0.447,NA,188,5.0,69.2,2.5,9.5,10155
2010,Josh Hamilton,TEX,29,133,571,32,95,100,8,⋯,0.411,0.633,0.445,NA,175,4.9,55.3,5.9,8.4,1875
2020,Marcell Ozuna,ATL,29,60,267,18,38,56,0,⋯,0.431,0.636,0.444,NA,178,-0.2,26.4,-8.1,2.5,10324
2011,Jose Bautista,TOR,30,149,655,43,105,103,9,⋯,0.447,0.608,0.443,NA,180,4.9,66.3,-11.1,8.1,1887


In [10]:
war <- data %>%
group_by(Age) %>%
summarize(avg_WAR = mean(WAR))
war
## Prime for WAR is 27-28 disregarding the extreme young ages. 
## Batters who are called up at a young age are generally very productive. Likely because if they are called up at such a 
## young age they are generational talents. There is a very small sample size for ages 19-22 that can be skewed
## easily by monster season such as Bryce Harper in 2015. 
## WAR decreases dramatically with age .. likely due to regression on defence and athletic ability. 

Age,avg_WAR
<int>,<dbl>
19,4.400000
20,4.483333
21,2.994118
22,3.340000
23,2.858108
24,2.881308
25,2.740741
26,2.897326
27,3.090099


In [11]:
ops <- data %>%
mutate(ops = OBP + SLG) %>%
group_by(Age) %>%
summarize(avg_ops = mean(ops))
ops
## However OPS is fairly constant throughout with a minor dip from 30-35. Players over 35 have strong OPS numbers, likely as
## players who are playing past 35 are likely very accomplished batters. 
## Once again the very young ages produce the best OPS, likely following the same reasoning as previously mentioned. 

Age,avg_ops
<int>,<dbl>
19,0.8170000
20,0.8390000
21,0.8284118
22,0.8000250
23,0.7730135
24,0.7772150
25,0.7807160
26,0.7840588
27,0.7905594


In [12]:
hr <- data %>%
group_by(Age) %>%
summarize(avg_hr = mean(HR))
hr
## Similar to most offensive statistics, home runs seem to generally peak from age 28-30
## Once again young ages are positively skewed. 

Age,avg_hr
<int>,<dbl>
19,22.00000
20,19.00000
21,20.05882
22,20.62500
23,17.95946
24,18.68224
25,18.31481
26,18.91979
27,18.68317


In [27]:
pa <- data %>%
group_by(Age) %>%
summarize(avg_pa = mean(PA))
pa
## Plate Appearances seems to peak around 34-35.. however they vary much more than offensive statistics. 

Age,avg_pa
<int>,<dbl>
19,597.0000
20,608.5000
21,551.8824
22,595.5250
23,561.2297
24,575.8131
25,552.0000
26,574.1390
27,555.3564


In [45]:
## There seems to be a spike at age 35 for most statistics. Time to analyze the individual data rows comparative to the
## general data. 

thfive <- data %>%
filter(Age == 35)
num_thfive <- thfive %>% select(where(is.numeric))
num_data <- data %>% select(where(is.numeric))
colMeans(num_thfive)
colMeans(num_data)

## It appears that 35 year old batters are generally more effective in all aspects of hitting, but most prominently in terms
## of power production. The difference between slugging percentage and home runs is the most significant difference between
## 35 year old and the entire field. However the defence of 35 year old is far below the average, explaining the negative
## difference in WAR. 

Season          Age            G           PA           HR            R 
2015.3170732   35.0000000  141.0731707  589.4878049   21.6585366   74.4390244 
         RBI           SB          ISO        BABIP          AVG          OBP 
  77.2439024    5.1463415    0.1843415    0.2930244    0.2711707    0.3443659 
         SLG         wOBA         wRC.          BsR          Off          Def 
   0.4555122    0.3443659  115.8536585   -2.0243902    9.2268293   -5.5170732 
         WAR     playerid 
   2.3902439 3402.6585366

Season          Age            G           PA           HR            R 
2015.9301192   28.5146262  136.9582882  567.5715060   18.9609967   72.2226436 
         RBI           SB          ISO        BABIP          AVG          OBP 
  69.2502709    9.3022752    0.1773044    0.3048602    0.2688261    0.3383104 
         SLG         wOBA         wRC.          BsR          Off          Def 
   0.4461360    0.3380699  112.1554713    0.2552546    8.9007584   -1.1812568 
         WAR     playerid 
   2.7121885 8330.8694475